<a href="https://colab.research.google.com/github/pkwbim/ai-learning/blob/master/poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = 'https://github.com/Elwing-Chou/ml0716/raw/master/poem_test.csv'
urlretrieve(url, 'poem_test.csv')

url = 'https://github.com/Elwing-Chou/ml0716/raw/master/poem_train.csv'
urlretrieve(url, 'poem_train.csv')

('poem_train.csv', <http.client.HTTPMessage at 0x7f5edb2dc940>)

In [ ]:
import pandas as pd
train_df = pd.read_csv('poem_train.csv', encoding="utf-8")
test_df = pd.read_csv('poem_test.csv', encoding="utf-8")

In [ ]:
u = train_df['作者'].unique()
trans = { n:i for i, n in enumerate(u)}
trans_r = { i:n for i, n in enumerate(u)}
y_train = train_df['作者'].replace(trans)
y_test = test_df['作者'].replace(trans)

In [ ]:
# demo
# iloc: [1st, 2nd, 3rd, ...]
# 檢查標點和換行是不是有去掉
import jieba
p = train_df.iloc[0]['內容']
' '.join(jieba.cut(p))

'平林 漠漠 煙如織 ， 寒山 一帶 傷心 碧 。 \r\n 暝 色入 高樓 ， 有人 樓上 愁 。 玉階空 佇立 ， 宿鳥 歸 飛急 。 \r\n 何處 是 歸程 ？ 長 亭 連短亭 。 ( 連短亭   一作 ： 更 短亭 )   \r\n                '

In [46]:
# Series.apply(func name)
import re
def poemcut(p):
  p = re.sub(r'[，：。？\n\r]', r' ', p)
  return ' '.join(jieba.cut(p))
x_train = train_df['內容'].apply(poemcut)
x_test = test_df['內容'].apply(poemcut)
x_test


0      日照 香爐生 紫煙   遙看 瀑布 掛 前川   飛流 直下 三千尺   疑是 銀河 落九天  
1     朝辭 白帝 彩雲間   千裡 江陵 一日 還   兩岸 猿聲 啼 不住   輕舟 已過 萬 ...
2      李白 乘舟 將欲行   忽聞 岸上 踏歌 聲   桃花潭水 深 千尺   不及 汪倫送 我情  
3       故人 西辭黃鶴樓   煙花 三月 下揚州   孤帆 遠影 碧空 儘   唯見長 江天 際流  
4                 危樓 高 百尺   手可摘 星辰   不敢 高聲語   恐驚 天上 人  
5                  床前 明月光   疑是 地上 霜   舉頭 望明月   低頭思 故鄉  
6     天門 中斷 楚江 開   碧水 東流 至此 回   兩岸 青山 相對 出   孤帆 一片 日...
7               眾鳥 高 飛 儘   孤雲獨 去 閒   相看 兩不厭   隻 有 敬亭山  
8     鳳凰 台上 鳳凰遊   鳳去 台空江 自流   吳宮 花草 埋 幽徑   晉代 衣冠 成古丘...
9     渡遠 荊門外   來 從 楚國遊   山 隨 平野 儘   江入 大荒 流   月 下 飛天...
10    百川 日東流   客去 亦 不息   我 生苦 漂 蕩   何時 有 終極   讚 公 釋 ...
11    細泉 兼 輕冰   沮洳 棧道 濕   不辭 辛苦 行   迫此 短景急   石門 雪雲隘 ...
12    首路 栗亭 西   尚想 鳳凰村   季冬 攜 童稚   辛苦 赴 蜀門   南登 木皮 嶺...
13    落日 在 簾 鉤   溪邊 春事幽   芳菲 緣岸 圃   樵 爨 倚灘 舟   啅 雀 爭...
14    竇侍 禦   驥 之子   鳳之雛   年 未 三十 忠義俱   骨鯁 絕代 無   炯 如...
15    百草 競春華   麗春應 最勝   少須 好 顏色   多漫枝條 剩   紛紛 桃李 枝  ...
16    何年 顧虎頭   滿壁畫 瀛州   赤 日 石林 氣   青天 江海 流   錫飛常 近鶴 ...
17    野寺 隱喬木   山僧 高下 居   石門 日色異   絳 氣橫 扶疏   窈窕 入

In [ ]:
# 測試資料不要重做fit,如果你的詞不在常初fit的欄位，則去掉
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
x_train_count

<2731x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 85677 stored elements in Compressed Sparse Row format>

In [43]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_count, y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [44]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)


0.8

In [49]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat, 
      columns=[ n+'(預測值)'for n in u],
      index=[n+'(真實)' for n in u])


,李白(預測值),杜甫(預測值),白居易(預測值)
李白(真實),8,1,1
杜甫(真實),1,8,1
白居易(真實),1,1,8


In [64]:
test = '我打江南走過  那等在季節里的容顏如蓮花的開落  東風不來，三月的柳絮不飛  你底心如小小的寂寞的城  恰若青石的街道向晚  跫音不響，三月的春帷不揭  你底心是小小的窗扉緊掩  我達達的馬蹄是美麗的錯誤  我不是歸人，是個過客…… '
count = vec.transform([poemcut(test)])
answer = clf.predict(count)[0]
proba = clf.predict_proba(count)[0]
for name, p in zip(u, proba):
  print(name, '的機率：', p)
print(proba)
print('Ans:', trans_r[answer])

李白 的機率： 0.22496514882715682
杜甫 的機率： 0.6997692416013379
白居易 的機率： 0.07526560957151852
[0.22496515 0.69976924 0.07526561]
Ans: 杜甫


In [59]:
list(zip([1,2,3], [4,5,6]))

[(1, 4), (2, 5), (3, 6)]